# Finding the nearest train/tube station

The purpose of this project is to create module that takes a London postcode input e.g. N4 4AF and will output the nearest train station to it.

In [1]:
import pandas as pd
import requests
from geopy import distance
import re
from collections import namedtuple

In [3]:
from pip._internal.utils.misc import get_installed_distributions
import sys
#import numpy as np # imported to test whether numpy shows up, which it does!

def get_imported_packages():
    p = get_installed_distributions()
    p = {package.key:package.version for package in p}

    imported_modules = set(sys.modules.keys())
    
    imported_modules.remove('pip')

    modules = [(m, p[m]) for m in imported_modules if p.get(m, False)]

    return modules


def generate_requirements(filepath:str, modules):
    with open(filepath, 'w') as f:
        for module, version in modules:
            f.write(f"{module}=={version}\n")


generate_requirements('requirements.txt', get_imported_packages())

In [2]:
# A csv file of london stations can be downloaded from this link
# https://www.doogal.co.uk/london_stations.php

# TODO join station data to tube line data using TFL api to get info
# Also replace hard coded file with retrieving lat/lon from TFL api
london_stations_df = pd.read_csv('london_stations.csv')

In [2]:
# LONDON_STATIONS = tfl_api.LONDON_STATIONS

In [3]:
Station = namedtuple('Station', ['name', 'lat_lon'])

In [5]:
# TODO change to tfl_api.LONDON_STATIONS
LONDON_STATIONS = [Station(s[0], (s[1], s[2])) for s in london_stations_df[['Station', 'Latitude', 'Longitude']].values]
# for x in london_stations_df[['Station', 'Latitude', 'Longitude']].values:
#     LONDON_STATIONS.append((x[0],(x[1],x[2])))



In [6]:
LONDON_STATIONS[0]

Station(name='Abbey Road', lat_lon=(51.5319519867171, 0.00372337109717115))

In [7]:
# TODO refactor - use less code
# TODO Add documentation
class NearestStation:
        
    def __init__(self, postcode, unit='m', stations=LONDON_STATIONS):
        self.postcode = postcode.lower()
        self.lat_lon = self.geo_dict['lat_lon']
        self._unit = unit
        self.stations = stations
     
    @staticmethod
    def extract_postcode(postcode):
        postcode = postcode
        regex_str = r'[A-Z]{1,2}[0-9R][0-9A-Z]? [0-9][A-Z]{2}'
        return re.findall(regex_str, postcode, flags=re.IGNORECASE)[0]
    
    @staticmethod
    def nearest_station(lat_lon, unit='m', n=1, stations=LONDON_STATIONS):
        if unit == 'm':
            distances = [(s.name, round(distance.distance(lat_lon, s.lat_lon).miles,1)) for s in stations]
            distances.sort(key=lambda x:x[1])
            return distances[:n]
        elif unit == 'km':
            distances = [(s.name, round(distance.distance(lat_lon, s.lat_lon).km,1)) for s in stations]
            distances.sort(key=lambda x:x[1])
            return distances[:n]
    
    @property
    def unit(self):
        return self._unit
    
    @unit.setter
    def unit(self, new_unit):
        if new_unit in ['m', 'km']:
            self._unit = new_unit
        else:
            raise ValueError('Enter "m" for miles or "km" for kilometers')
        
    @property
    def endpoint(self):
        pc = self.postcode.replace(' ','')
        return f'http://api.postcodes.io/postcodes/{pc}'
    
    @property
    def geo_dict(self):
        geocoded_dict = {}
        r = requests.get(self.endpoint)
        geocoded_dict['postcode'] = r.json()['result']['postcode']
        geocoded_dict['lat_lon'] = (r.json()['result']['latitude'], r.json()['result']['longitude'])
        return geocoded_dict
    
    @property
    def distances(self):
        if self._unit == 'm':
            distances = [(s.name, round(distance.distance(self.lat_lon, s.lat_lon).miles,1)) for s in self.stations]
            distances.sort(key=lambda x:x[1])
            return distances
        elif self._unit == 'km':
            distances = [(s.name, round(distance.distance(self.lat_lon, s.lat_lon).km,1)) for s in self.stations]
            distances.sort(key=lambda x:x[1])
            return distances
        
    def find_nearest_station(self): 
        return (self.distances[0])

    def find_nearest_n_stations(self,n):
        return (self.distances[:n])
    
    def find_stations_with_x_radius(self, n):
        return [x for x in self.distances if x[1] <= n]

In [8]:
NearestStation.nearest_station(('51.558033', '-0.162331'), n=5)

[('Hampstead Heath', 0.2),
 ('Belsize Park', 0.5),
 ('Gospel Oak', 0.5),
 ('Hampstead', 0.7),
 ('Chalk Farm', 1.0)]

In [14]:
NearestStation.extract_postcode('48 antil road n17 3ss')

'n17 3ss'

In [6]:
x = NearestStation('N4 4AF')

In [7]:
x.find_nearest_station()

('Harringay', 0.3)

In [8]:
x.find_nearest_n_stations(5)

[('Harringay', 0.3),
 ('Harringay Green Lanes', 0.4),
 ('Crouch Hill', 0.5),
 ('Manor House', 0.5),
 ('Finsbury Park', 0.6)]

In [9]:
x.unit

'm'

In [10]:
x.find_stations_with_x_radius(1)

[('Harringay', 0.3),
 ('Harringay Green Lanes', 0.4),
 ('Crouch Hill', 0.5),
 ('Manor House', 0.5),
 ('Finsbury Park', 0.6),
 ('Arsenal', 1.0),
 ('Hornsey', 1.0)]

In [11]:
test_postcodes = ['N15 3AD', 'N4 4AF', 'N10 3QS', 'N15 4AR', 'N15 4JF']

In [12]:
for postcode in test_postcodes:
    obj = NearestStation(postcode)
    print(obj.postcode)
    print(obj.find_stations_with_x_radius(1))
    print('------------------')

n15 3ad
[('Harringay Green Lanes', 0.5), ('Turnpike Lane', 0.6), ('Harringay', 0.7), ('Hornsey', 0.8), ('Manor House', 0.9), ('Seven Sisters', 0.9), ('South Tottenham', 0.9), ('Stamford Hill', 0.9)]
------------------
n4 4af
[('Harringay', 0.3), ('Harringay Green Lanes', 0.4), ('Crouch Hill', 0.5), ('Manor House', 0.5), ('Finsbury Park', 0.6), ('Arsenal', 1.0), ('Hornsey', 1.0)]
------------------
n10 3qs
[('Highgate', 0.8), ('East Finchley', 0.9)]
------------------
n15 4ar
[('Seven Sisters', 0.3), ('Tottenham Hale', 0.3), ('Bruce Grove', 0.5), ('South Tottenham', 0.5), ('Stamford Hill', 1.0)]
------------------
n15 4jf
[('Bruce Grove', 0.3), ('Seven Sisters', 0.4), ('South Tottenham', 0.6), ('Tottenham Hale', 0.6)]
------------------
